In [1]:
from bs4 import BeautifulSoup
import requests
import re
from urllib.parse import urljoin, urlparse
import pandas as pd
import numpy as np

In [2]:
# THIS CODE WORKS FOR COHORTS FROM 2019-2022
# Website of job market candidates that we want to scrape
# Note the year, as it will be used to save the CSV output later:
# 2017
url = "https://web.archive.org/web/20170302034545/https://economics.yale.edu/graduate/graduate-placement"

# Get HTML content of the website
page = requests.get(url)

soup = BeautifulSoup(page.text, "html.parser")
#print(soup)

# Create a list to store the data
data = []

# Initialize lists
names = []
cv_links = []
personal_websites = []
dissertations = []
fields_of_interest = []
committee_members = []

In [121]:
# soup

In [3]:
# NAMES
headers = soup.find_all("h3")
for header in headers:
    name = header.get_text(strip=True)
    names.append(name)

# CV LINKS & PERSONAL WEBSITES

# Find all links
atags = soup.find_all('a')
atags

for atag in atags:
    if 'CV' in atag.get_text():
        # Extract link from the tag
        cv_link = atag.get("href")
        # Append the link to the list
        cv_links.append(cv_link)
    
    if 'Website' in atag.get_text():
        # Extract link from the tag
        personal_website = atag.get('href')
        # Append the link to the list
        personal_websites.append(personal_website)

In [4]:
# Use a dictionary for uniqueness and order
dissertations_dict = {}

# Find all <strong> tags with any sibling text
strongtags = soup.find_all('strong')

for strongtag in strongtags:
    text = strongtag.get_text(strip=True) # Clean whitespace
    # DISSERTATION TITLES
    if 'Dissertation Title' in text:
        dissertation_title = strongtag.next_sibling.strip()
        dissertations_dict[dissertation_title] = None
    elif 'Job Market Paper' in text:
        jm_paper = strongtag.next_sibling.strip()
        dissertations_dict[jm_paper] = None

# Convert to list while preserving order
dissertations = list(dissertations_dict.keys())

In [5]:
areas = soup.find_all('strong', string=lambda text: text and 'Areas of Concentration' in text)

for area in areas:
    text = area.get_text(strip=True)  # Clean whitespace
    if 'Areas of Concentration:' in text:
        # Safely navigate siblings
        sibling = area.next_sibling
        # Skip empty or invalid siblings
        while sibling and not sibling.strip():  
            sibling = sibling.next_sibling
        # Add to list if valid
        if sibling:
            field = sibling.strip()
            fields_of_interest.append(field)

# Print the results to verify
fields_of_interest


TypeError: 'NoneType' object is not callable

In [ ]:
# areas = soup.find_all('strong', string=lambda text: text and 'Areas of Concentration' in text)

# for area in areas:
#     text = area.get_text(strip=True) # Clean whitespace
#     # AREAS OF CONCENTRATION
#     if 'Areas of Concentration:' in text:
#         field = area.next_sibling.strip()
#         fields_of_interest.append(field)

In [ ]:
# # Use a dictionary for uniqueness and order
# dissertations_dict = {}
# fields_of_interest_dict = {} #WAIT! I don't want this to be a dictionary because there's repeat values!!

# # Find all <strong> tags with any sibling text
# strongtags = soup.find_all('strong')

# for strongtag in strongtags:
#     text = strongtag.get_text(strip=True) # Clean whitespace
#     # DISSERTATION TITLES
#     if 'Dissertation Title' in text:
#         dissertation_title = strongtag.next_sibling.strip()
#         dissertations_dict[dissertation_title] = None
#     elif 'Job Market Paper' in text:
#         jm_paper = strongtag.next_sibling.strip()
#         dissertations_dict[jm_paper] = None
    
#         # FIELDS OF INTEREST
#     if 'Areas of Concentration:' in text:
#         field = strongtag.next_sibling.get_text(strip=True)
#         fields_of_interest_dict[field] = None

# # Convert to list while preserving order
# dissertations = list(dissertations_dict.keys())
# fields_of_interest = list(fields_of_interest_dict.keys())

# fields_of_interest


In [ ]:
# # FIELDS OF INTEREST, DISSERTATIONS

# # Create a set to store unique values temporarily
# dissertations_set = set() # set() removes duplicate values
# fields_of_interest_set = set()

# # Find all <strong> tags with any sibling text
# strongtags = soup.find_all('strong')

# for strongtag in strongtags:
#     text = strongtag.get_text(strip=True)  # Clean whitespace
#     # DISSERTATION TITLES
#     if 'Dissertation Title' in text:
#         dissertation = strongtag.next_sibling.strip()
#         dissertations_set.add(dissertation)  # Add to set for uniqueness
#     elif 'Job Market Paper' in text:
#         dissertation = strongtag.next_sibling.strip()
#         dissertations_set.add(dissertation)  # Add to set for uniqueness
#     # FIELDS OF INTEREST
#     elif 'Areas of Concentration' in text:
#         field = strongtag.next_sibling.strip()
#         fields_of_interest_set.add(field)
        

# # Convert the set back to a list
# dissertations = list(dissertations_set)
# fields_of_interest = list(fields_of_interest_set)

# len(dissertations)

# # print(len(dissertations),
# # len(fields_of_interest),
# # len(headers))

In [125]:
# DEBUGGING

# Find the maximum length of the lists
max_length = max(len(names), len(dissertations), len(fields_of_interest), len(cv_links), len(personal_websites))

# Pad the lists with None if they are shorter than max_length
names += [None] * (max_length - len(names))
dissertations += [None] * (max_length - len(dissertations))
fields_of_interest += [None] * (max_length - len(fields_of_interest))
cv_links += [None] * (max_length - len(cv_links))
personal_websites += [None] * (max_length - len(personal_websites))

print(len(names),
    len(dissertations),
    len(fields_of_interest),
    len(cv_links),
    len(personal_websites))

headers

9 9 9 9 9


[<h3 class="field-content">Yaniv Ben-Ami</h3>,
 <h3 class="field-content">Gregory Cox</h3>,
 <h3 class="field-content">Sharat Ganapati</h3>,
 <h3 class="field-content">Matthew Grant</h3>,
 <h3 class="field-content">Chiara Margaria</h3>,
 <h3 class="field-content">Craig Palsson</h3>,
 <h3 class="field-content">Meredith Startz</h3>,
 <h3 class="field-content">Haobin Wang</h3>,
 <h3 class="field-content">Jeffrey Weaver</h3>]

In [126]:
df = pd.DataFrame({
    "name": names,
    "school": 'Yale',
    "year": '2017',
    'cv_link': cv_links,
    'personal_website': personal_websites,
    'dissertation': dissertations,
    'field': fields_of_interest,
})
df

,name,school,year,cv_link,personal_website,dissertation,field
0,Yaniv Ben-Ami,Yale,2017,https://web.archive.org/web/20170302034545/htt...,None,,None
1,Gregory Cox,Yale,2017,https://web.archive.org/web/20170302034545/htt...,None,None,None
2,Sharat Ganapati,Yale,2017,https://web.archive.org/web/20170302034545/htt...,None,None,None
3,Matthew Grant,Yale,2017,https://web.archive.org/web/20170302034545/htt...,None,None,None
4,Chiara Margaria,Yale,2017,https://web.archive.org/web/20170302034545/htt...,None,None,None
5,Craig Palsson,Yale,2017,https://web.archive.org/web/20170302034545/htt...,None,None,None
6,Meredith Startz,Yale,2017,https://web.archive.org/web/20170302034545/htt...,None,None,None
7,Haobin Wang,Yale,2017,https://web.archive.org/web/20170302034545/htt...,None,None,None
8,Jeffrey Weaver,Yale,2017,https://web.archive.org/web/20170302034545/htt...,None,None,None


In [101]:
# Identify and flag empty cells
empty_cells = df.isnull()
empty_rows = df[empty_cells.any(axis=1)]

print("\nRows with empty cells:")
print(len(empty_rows)) # Length should be zero, but it's 6
print(empty_rows)


Rows with empty cells:
1
          name school  year cv_link personal_website  \
19  Ling Zhong   Yale  2019    None             None   

                         dissertation  \
19  "Three Essays in Labor Economics"   

                                                field  
19  Labor Economics, Economics of Education, Publi...  


In [102]:
# Save the data to a CSV file, making sure to specify the year
df.to_csv("yale_econ_jm_candidates_2017.csv", index=False)

#Check first few columns
print(df.head())

               name school  year  \
0     Shameel Ahmad   Yale  2019   
1    Antoine Arnoud   Yale  2019   
2     Leila Bengali   Yale  2019   
3  Anna Bykhovskaya   Yale  2019   
4    Jin-Wook Chang   Yale  2019   

                                             cv_link  \
0  https://web.archive.org/web/20190329023737/htt...   
1  https://web.archive.org/web/20190329023737/htt...   
2  https://web.archive.org/web/20190329023737/htt...   
3  https://web.archive.org/web/20190329023737/htt...   
4  https://web.archive.org/web/20190329023737/htt...   

                                    personal_website  \
0  https://web.archive.org/web/20190329023737/htt...   
1  https://web.archive.org/web/20190329023737/htt...   
2  https://web.archive.org/web/20190329023737/htt...   
3  https://web.archive.org/web/20190329023737/htt...   
4  https://web.archive.org/web/20190329023737/htt...   

                                        dissertation  \
0     "Demography and Development in Colonial India" 

In [ ]:
# DEBUGGING 2021-- KEEP WORKING ON THIS
# Manually drop duplicate CV link for Lopez-Pena
# df = df.drop(df.loc[df['cv_link'] == 'https://web.archive.org/web/20210228201841/https://www.dropbox.com/s/c5awtuhrdvno4al/cv_lopezpena_201102.pdf?dl=0'].index)